In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io

In [3]:
# Load the dataset

from google.colab import files
uploaded = files.upload()

Saving cleaned_combined_crypto_data .csv to cleaned_combined_crypto_data .csv


In [10]:
 # Read the file into a pandas DataFrame
df = pd.read_csv("cleaned_combined_crypto_data .csv")

In [12]:
#Adding a Spread Column
# to calculate the price spread for each pair

df['Spread'] = df['High'] - df['Low']

In [13]:
print(df.head())

        Pair     High      Low  Last Price        Volume  \
0    Cardano     1.09     0.99        1.06  1.965847e+07   
1   Polkadot     7.20     6.56        7.09  3.324909e+05   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04   
3  Chainlink    22.15    20.11       21.76  2.528274e+05   
4   Litecoin   118.76   100.79      115.96  3.331147e+04   

                       Timestamp  Spread  
0  2025-01-16 00:18:20.000000000    0.10  
1  2025-01-16 00:18:20.000000000    0.64  
2  2025-01-16 00:18:20.000000000  286.41  
3  2025-01-16 00:18:20.000000000    2.04  
4  2025-01-16 00:18:20.000000000   17.97  


In [14]:
#Calculating the position of the Last Price within the range
df['Position'] = (df['Last Price'] - df['Low']) / (df['High'] - df['Low'])


In [15]:
print(df.head())

        Pair     High      Low  Last Price        Volume  \
0    Cardano     1.09     0.99        1.06  1.965847e+07   
1   Polkadot     7.20     6.56        7.09  3.324909e+05   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04   
3  Chainlink    22.15    20.11       21.76  2.528274e+05   
4   Litecoin   118.76   100.79      115.96  3.331147e+04   

                       Timestamp  Spread  Position  
0  2025-01-16 00:18:20.000000000    0.10  0.700000  
1  2025-01-16 00:18:20.000000000    0.64  0.828125  
2  2025-01-16 00:18:20.000000000  286.41  0.842464  
3  2025-01-16 00:18:20.000000000    2.04  0.808824  
4  2025-01-16 00:18:20.000000000   17.97  0.844185  


In [16]:
#To Compute the percentage volatility of each cryptocurrency

df['Volatility'] = ((df['High'] - df['Low']) / df['Low']) * 100


In [17]:
print(df.head())

        Pair     High      Low  Last Price        Volume  \
0    Cardano     1.09     0.99        1.06  1.965847e+07   
1   Polkadot     7.20     6.56        7.09  3.324909e+05   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04   
3  Chainlink    22.15    20.11       21.76  2.528274e+05   
4   Litecoin   118.76   100.79      115.96  3.331147e+04   

                       Timestamp  Spread  Position  Volatility  
0  2025-01-16 00:18:20.000000000    0.10  0.700000   10.101010  
1  2025-01-16 00:18:20.000000000    0.64  0.828125    9.756098  
2  2025-01-16 00:18:20.000000000  286.41  0.842464    8.987921  
3  2025-01-16 00:18:20.000000000    2.04  0.808824   10.144207  
4  2025-01-16 00:18:20.000000000   17.97  0.844185   17.829150  


In [19]:
#Extracting features like hour and minute from the Timestamp column

# Convert 'Timestamp' column to datetime objects
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute

print(df.head())

        Pair     High      Low  Last Price        Volume           Timestamp  \
0    Cardano     1.09     0.99        1.06  1.965847e+07 2025-01-16 00:18:20   
1   Polkadot     7.20     6.56        7.09  3.324909e+05 2025-01-16 00:18:20   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04 2025-01-16 00:18:20   
3  Chainlink    22.15    20.11       21.76  2.528274e+05 2025-01-16 00:18:20   
4   Litecoin   118.76   100.79      115.96  3.331147e+04 2025-01-16 00:18:20   

   Spread  Position  Volatility  Hour  Minute  
0    0.10  0.700000   10.101010     0      18  
1    0.64  0.828125    9.756098     0      18  
2  286.41  0.842464    8.987921     0      18  
3    2.04  0.808824   10.144207     0      18  
4   17.97  0.844185   17.829150     0      18  


In [21]:
#Computing rolling averages for prices or volume over a time window:

df['Rolling_Avg_Volume'] = df['Volume'].rolling(window=3).mean()

print(df.head())

        Pair     High      Low  Last Price        Volume           Timestamp  \
0    Cardano     1.09     0.99        1.06  1.965847e+07 2025-01-16 00:18:20   
1   Polkadot     7.20     6.56        7.09  3.324909e+05 2025-01-16 00:18:20   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04 2025-01-16 00:18:20   
3  Chainlink    22.15    20.11       21.76  2.528274e+05 2025-01-16 00:18:20   
4   Litecoin   118.76   100.79      115.96  3.331147e+04 2025-01-16 00:18:20   

   Spread  Position  Volatility  Hour  Minute  Rolling_Avg_Volume  
0    0.10  0.700000   10.101010     0      18                 NaN  
1    0.64  0.828125    9.756098     0      18                 NaN  
2  286.41  0.842464    8.987921     0      18        6.668551e+06  
3    2.04  0.808824   10.144207     0      18        2.000046e+05  
4   17.97  0.844185   17.829150     0      18        1.002781e+05  


In [23]:
#Apply log transformation to the Volume column to reduce skewness

df['Log_Volume'] = np.log1p(df['Volume'])

print(df.head())

        Pair     High      Low  Last Price        Volume           Timestamp  \
0    Cardano     1.09     0.99        1.06  1.965847e+07 2025-01-16 00:18:20   
1   Polkadot     7.20     6.56        7.09  3.324909e+05 2025-01-16 00:18:20   
2   Ethereum  3473.02  3186.61     3427.90  1.469543e+04 2025-01-16 00:18:20   
3  Chainlink    22.15    20.11       21.76  2.528274e+05 2025-01-16 00:18:20   
4   Litecoin   118.76   100.79      115.96  3.331147e+04 2025-01-16 00:18:20   

   Spread  Position  Volatility  Hour  Minute  Rolling_Avg_Volume  Log_Volume  
0    0.10  0.700000   10.101010     0      18                 NaN   16.794019  
1    0.64  0.828125    9.756098     0      18                 NaN   12.714371  
2  286.41  0.842464    8.987921     0      18        6.668551e+06    9.595360  
3    2.04  0.808824   10.144207     0      18        2.000046e+05   12.440466  
4   17.97  0.844185   17.829150     0      18        1.002781e+05   10.413687  


In [25]:
#Checking for missing values

print(df.isnull().sum())

Pair                  0
High                  0
Low                   0
Last Price            0
Volume                0
Timestamp             0
Spread                0
Position              0
Volatility            0
Hour                  0
Minute                0
Rolling_Avg_Volume    2
Log_Volume            0
dtype: int64


In [26]:
#dropping the coloumns with NaN values
df = df.dropna(axis=1)

In [28]:
print(df.isnull().sum())

Pair          0
High          0
Low           0
Last Price    0
Volume        0
Timestamp     0
Spread        0
Position      0
Volatility    0
Hour          0
Minute        0
Log_Volume    0
dtype: int64
